In [42]:
import tensorflow as tf
import numpy as np
import random
import os
import json
from tqdm import tqdm
import os
import sys
from collections import defaultdict

In [8]:
%cd ..
%cd ..
%ls

/shared/hltdir1/disk1/home/max/code/umls-embeddings/python/eukg
/shared/hltdir1/disk1/home/max/code/umls-embeddings/python
bin/  data/  eukg/  __init__.py  __pycache__/  results.txt  Untitled.ipynb


In [9]:
from eukg.data import data_util, TfDataGenerator
from eukg import Config, train
from eukg.emb import AceModel
from eukg.tf_util import checkpoint_utils

W1228 20:38:26.784793 139718646130496 deprecation_wrapper.py:119] From /shared/hltdir1/disk1/home/max/code/umls-embeddings/python/eukg/emb/AceModel.py:30: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.



In [12]:
class TestConfig:
  def __init__(self):
    pass

config = TestConfig()
config.val_proportion = 0.1
config.embedding_size = 50
config.energy_norm_ord = 1
config.seed = 1337
config.mode = 'disgen'
config.model = 'transd-distmult'
config.run_name = 'transd-dm-disgen-ace-7'
config.ace_model = True
config.no_semantic_network = True
config.train_bert = False
config.learning_rate = 1e-5
config.batch_size = 16
config.val_batch_size = 16
config.num_epochs = 100
config.data_dir = '/users/max/data/artifacts/umls-embeddings'
config.secondary_data_dir = '/users/max/data/artifacts/umls-embeddings-compressed'
config.model_dir = '/users/max/data/models/umls-embeddings'
config.summaries_dir = '/shared/hltdir4/disk1/max/logs'
config.eval_mode = 'save'
config.eval_dir = '/users/max/data/artifacts/umls-embeddings'
config.load_embeddings = True 
config.embedding_file = '/users/max/data/artifacts/umls-embeddings/transd-dm-disgen-ace-7/test_embeddings.npz'
config.load = True
config.num_workers = 6
config.buffer_size = 1
config.gpu_memory_growth = True
config.num_generator_samples = 30
config.lm_encoder_size = 768

In [34]:
# Invalid examples from eukg.test.ppa when using saved embeddings
# ((106346, 33264, 124316, 3.1119406), (106346, 33264, 1536913, 2.8550181))
# (('C0489903', 'disease_excludes_primary_anatomic_site', 'C0861214', '3.1119406'), ('C0489903', 'disease_excludes_primary_anatomic_site', 'C4496967', '2.8550181'))

# ((57302, 635, 55641, 2.7145562), (57302, 635, 652367, 2.682478))
# (('C0028027', 'has_active_ingredient', 'C4522948', '2.7145562'), ('C0028027', 'has_active_ingredient', 'C4756991', '2.682478'))

# ((721864, 26418, 502328, 3.1027548), (721864, 26418, 1045134, 3.074905))
# (('C1261322', 'has_method', 'C0522855', '3.1027548'), ('C1261322', 'has_method', 'C2754938', '3.074905'))

# ((209953, 19305, 23896, 3.8642027), (209953, 19305, 1552713, 3.7970138))
# (('C1511632', 'gene_product_plays_role_in_biological_process', 'C1450798', '3.8642027'), ('C1511632', 'gene_product_plays_role_in_biological_process', 'C4692589', '3.7970138'))

# ((192966, 992, 599506, 3.7793498), (192966, 992, 1211380, 3.5460076))
# (('C0038636', 'has_inactive_ingredient', 'C3645296', '3.7793498'), ('C0038636', 'has_inactive_ingredient', 'C2013204', '3.5460076'))

# ((1017780, 62278, 1147921, 3.253294), (1017780, 62278, 282163, 3.210642))
# (('C1968275', 'contains', 'C4741856', '3.253294'), ('C1968275', 'contains', 'C3886221', '3.210642'))

# ((1120223, 167576, 1120219, 3.2748394), (1120223, 167576, 281781, 3.2267866))
# (('C4054219', 'concept_in_subset', 'C4054214', '3.2748394'), ('C4054219', 'concept_in_subset', 'C3254445', '3.2267866'))

# ((442237, 555, 58690, 2.8224788), (442237, 555, 315658, 2.8126345))
# (('C2345366', 'has_tradename', 'C0025646', '2.8224788'), ('C2345366', 'has_tradename', 'C0660621', '2.8126345'))

# ((747525, 21502, 747561, 3.4903193), (747525, 21502, 471601, 3.3539798))
# (('C0501933', 'sib_in_branch_of', 'C0501944', '3.4903193'), ('C0501933', 'sib_in_branch_of', 'C0561480', '3.3539798'))
outdir = os.path.join(config.eval_dir, config.run_name)
e_file = np.load(os.path.join(outdir, 'test_embeddings.npz'))

In [35]:
embs = e_file['embs']
embs_proj = e_file['p_embs']

In [36]:
def project(c, c_proj, r_proj):
  return c + np.sum(c * c_proj, axis=-1, keepdims=True) * r_proj

def dist(s, rel, o):
  h, h_proj = embs[s], embs_proj[s]
  r, r_proj = embs[rel], embs_proj[rel]
  t, t_proj = embs[o], embs_proj[o]
  
  return np.linalg.norm(project(h, h_proj, r_proj) + r - project(t, t_proj, r_proj),
            ord=1,
            axis=-1,
            keepdims=False)


In [40]:
cui2id, train_data, _, _ = data_util.load_metathesaurus_data(config.data_dir, 0.0)
test_data = data_util.load_metathesaurus_test_data(config.data_dir)

In [43]:
concepts = set()
sr2o = defaultdict(set)
or2s = defaultdict(set)
for s, r, o in zip(train_data['subj'], train_data['rel'], train_data['obj']):
  concepts.add(s)
  concepts.add(o)
  sr2o[(s, r)].add(o)
  or2s[(o, r)].add(s)

test_sr2o = defaultdict(set)
test_or2s = defaultdict(set)
for s, r, o in zip(test_data['subj'], test_data['rel'], test_data['obj']):
  concepts.add(s)
  concepts.add(o)
  test_sr2o[(s, r)].add(o)
  test_or2s[(o, r)].add(s)
  sr2o[(s, r)].add(o)
  or2s[(o, r)].add(s)

concepts = np.asarray(list(concepts), dtype=np.int32)

In [52]:
print(test_or2s.keys())

dict_keys([(463698, 1819), (54217, 992), (64745, 125), (42849, 992), (566372, 1), (1246389, 2559), (107873, 125), (221258, 1), (13037, 4614), (956515, 125), (1012159, 190891), (528299, 1), (628555, 2559), (740795, 2590), (249177, 557), (841754, 2590), (1503811, 125), (713934, 749832), (21541, 21502), (105896, 2590), (515227, 701695), (202010, 125), (108246, 2590), (1262794, 2559), (69292, 12529), (118667, 12531), (869405, 2590), (260188, 14477), (182453, 992), (1195474, 167576), (498197, 125), (18297, 73713), (278806, 992), (603476, 167576), (872776, 125), (437704, 15), (873394, 2590), (721176, 125), (1197912, 167576), (161971, 701695), (415874, 2559), (76067, 555), (268933, 715874), (29223, 637), (18686, 1819), (984083, 2591), (18295, 992), (767955, 5130), (535472, 679097), (85478, 163587), (49779, 637), (631814, 3847), (42698, 635), (1354271, 2559), (84019, 3750), (1451504, 125), (1192381, 167576), (133737, 3749), (90348, 11), (444952, 557), (300461, 4624), (237471, 15), (486624, 125

In [65]:
o = np.array([109688])
r = np.array([4614])
s = concepts

test_s = test_or2s[(int(o), int(r))]
all_s = or2s[(int(o), int(r))]
print(test_s)
rank = 1
ranks = []
for d, d_s in sorted(zip(dist(s, r, o), concepts), key=lambda x: x[0]):
  d_s = int(d_s)
  if d_s in test_s:
    ranks.append(((d_s, int(r), int(o)), rank, d))
    if len(ranks) == len(test_s):
      break
  elif d_s not in all_s:
    rank += 1
print(np.mean([x[1] for x in ranks]))

{1345410, 1344771, 1341187, 1342724, 1340549, 1345030, 1340936, 1340937, 1340298, 1345419, 1341190, 1344143, 1341456, 1339280, 1336466, 1341328, 1336468, 1340565, 1336854, 1344151, 1336216, 1340181, 1339161, 1337115, 1337106, 1339170, 1337123, 1341092, 1340325, 1338278, 1339559, 1345811, 1345321, 1345070, 1339695, 278066, 1341492, 1340087, 1339578, 1341503, 1344578, 1336514, 1342534, 1345222, 1344840, 1339722, 1337035, 1340751, 1339859, 1339092, 1338965, 1336789, 1345753, 1336154, 1342685, 1345758, 1340511, 1343968, 1340514, 1340515, 1340517, 1344487, 1338216, 1345768, 1341418, 1338988, 1342454, 1336438, 1336828, 1339133, 1344510, 1344639}
7143.055555555556


In [ ]:
random.seed(config.seed)
np.random.seed(config.seed)
all_models_dir = config.model_dir

config.model_dir = os.path.join(config.model_dir, config.model, config.run_name)

cui2id, train_data, _, _ = data_util.load_metathesaurus_data(config.data_dir, config.val_proportion)
id2cui = {v: k for k, v in cui2id.items()}
nrof_concepts = max(id2cui.keys()) + 1
print(f'{nrof_concepts} total unique concepts/rels')

data_generator = TfDataGenerator.TfTestDataGenerator(
  train_data,
  config.data_dir,
  config.secondary_data_dir,
  config.batch_size,
  config.lm_encoder_size,
  config.num_workers,
  config.buffer_size
)

if config.gpu_memory_growth:
  gpu_config = tf.ConfigProto()
  gpu_config.gpu_options.allow_growth = True
else:
  gpu_config = None

session = tf.Session(config=gpu_config)
if config.ace_model:
  ace_model = AceModel.ACEModel(config)
else:
  ace_model = None

model = train.init_model(config, data_generator, ace_model, test=True)

if config.pre_run_name is not None:
  pre_model_ckpt = tf.train.latest_checkpoint(
    os.path.join(all_models_dir, config.model, config.pre_run_name))
  checkpoint_utils.init_from_checkpoint(pre_model_ckpt)

tf.global_variables_initializer().run()
tf.local_variables_initializer().run()

# init saver
tf_saver = tf.train.Saver(max_to_keep=10)

if config.load:
  ckpt = tf.train.latest_checkpoint(config.model_dir)
  print('Loading checkpoint: %s' % ckpt)
  tf_saver.restore(session, ckpt)

# finalize graph
tf.get_default_graph().finalize()

outdir = os.path.join(config.eval_dir, config.run_name)
if not os.path.exists(outdir):
  os.makedirs(outdir)

batch_size = config.batch_size
emb_size = config.embedding_size
embeddings = np.zeros((nrof_concepts, emb_size), dtype=np.float32)
embeddings_proj = np.zeros((nrof_concepts, emb_size), dtype=np.float32)
seen_concepts = np.zeros(nrof_concepts, dtype=np.bool)
pbar = tqdm(total=nrof_concepts)
print('Creating concept embeddings...')
model.data_generator.load_concepts(session)
try:
  while True:
    (c_embs, c_embs_proj), c_ids = session.run([model.concept_embeddings, model.concept_ids])
    embeddings[c_ids] = c_embs
    embeddings_proj[c_ids] = c_embs_proj
    pbar.update(len(c_ids))
    seen_concepts[c_ids] = True
except tf.errors.OutOfRangeError:
  pass

print('Creating rel embeddings...')
model.data_generator.load_rels(session)
try:
  while True:
    (r_embs, r_embs_proj), r_ids = session.run([model.relation_embeddings, model.rel_ids])
    embeddings[r_ids] = r_embs
    embeddings_proj[r_ids] = r_embs_proj
    pbar.update(len(r_ids))
    seen_concepts[r_ids] = True
except tf.errors.OutOfRangeError:
  pass

seen_count = int(seen_concepts.astype(np.int32).sum())
total_count = len(seen_concepts)
print(f'Seen concepts: {seen_count}/{total_count}')
assert seen_count == total_count, 'Not all concepts have been seen!'
# print('Saving embeddings...')
# np.savez_compressed(
#   os.path.join(outdir, 'test_embeddings.npz'),
#   embs=embeddings,
#   p_embs=embeddings_proj
# )